In [21]:
import intake
import matplotlib.pyplot as plt
import xarray as xr
import dask

In [22]:
cat = intake.open_esm_datastore(
    'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json'
)

In [23]:
cat.search(long_name=['temperature*', 'Temperature*']).df.long_name.unique()

array(['temperature', 'reference height temperature',
       'minimum reference height temperature over output period',
       'maximum reference height temperature over output period',
       'surface temperature (radiative)',
       'atmospheric rain, after rain/snow repartitioning based on temperature',
       'atmospheric snow, after rain/snow repartitioning based on temperature',
       'daily maximum of average 2-m temperature',
       'potential temperature'], dtype=object)

In [24]:
# subset = cat.search(variable='TS',frequency='daily')
subset = cat.search(variable='TS',frequency='daily',forcing_variant='cmip6',experiment='ssp370')
subset.df

,variable,long_name,component,experiment,forcing_variant,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,TS,surface temperature (radiative),atm,ssp370,cmip6,daily,1.0,global,K,2015-01-01 12:00:00,2100-12-31 12:00:00,s3://ncar-cesm2-lens/atm/daily/cesm2LE-ssp370-...


In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = subset.to_dataset_dict()

In [35]:
dsets = subset.to_dataset_dict(
  zarr_kwargs={'consolidated': True, 'decode_times':True},
  aggregate=True,
  storage_options={'anon': 'True'}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


In [36]:
def subset_ds(ds):
    # Subset for TBOT and choose Boulder, Colorado's lat and lon and convert to degC
    da = ds.TS.sel(lat=40.015, lon=-105.2705, method='nearest') - 273.15
    da = da.groupby('time.year').mean()

    # Make sure that time is the format of a string
    da['year'] = da.year.astype(int)

    return da

In [37]:
dsets['atm.ssp370.daily.cmip6']

<xarray.Dataset>
Dimensions:    (member_id: 50, time: 31390, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) <U12 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    TS         (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 730, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.0
    host:                    mom1
    logname:                 sunseon
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    source:                  CAM
    time_period_freq:        day_1
    topography_file:         /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/f...
    intake_esm_varname:      ['TS']
    intake_esm_dataset_key:  atm.ssp370.daily.cmip6

In [38]:
da = subset_ds(dsets['atm.ssp370.daily.cmip6'])

In [ ]:
# Setup the figure
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot(111)

# Loop through and plot each member_id
for member_id in da.member_id.values:
    da.sel(member_id=member_id).plot(ax=ax, label=member_id)
    print(member_id)

# Limit the axis from 2015 to 2100
plt.xlim(2015, 2100)

# Increase the ticksize
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Add some labels and increase the size
plt.xlabel('Year', fontsize=16)
plt.ylabel('Temperature (degC)', fontsize=16)

# Add a legend
plt.legend(title='Member IDs', fontsize=12)

# Add a title
plt.title(
    'Experiment: SSP370 \n Forcing Variant: Smoothed Biomass \n Control Branch Year: 1301 \n 2015 - 2100',
    fontsize=20,
);


r10i1181p1f1
r10i1231p1f1
